##  Build a simple CNN model for classifying dog/cat  

Author: Anh Trung Tra   
Email: tratrunganh001@gmail.com

In [9]:
!pip install tensorflow-gpu==2.0.0-alpha0

  Using cached https://files.pythonhosted.org/packages/1a/66/32cffad095253219d53f6b6c2a436637bbe45ac4e7be0244557210dc3918/tensorflow_gpu-2.0.0a0-cp36-cp36m-manylinux1_x86_64.whl


In [10]:
import tensorflow as tf
print(tf.__version__)
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

keras = tf.keras
keras.backend.set_session(session)
%matplotlib inline

2.0.0-alpha0


/home/anhttra/Projects/gdg-mientrung-2019/venv/lib/python3.6/site-packages/tensorflow/python/client/session.py:1717: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [11]:
# Create a dictionary describing the features.  
image_feature_description = {
    'height': tf.io.FixedLenFeature([], tf.int64),
    'width': tf.io.FixedLenFeature([], tf.int64),
    'label': tf.io.FixedLenFeature([], tf.int64),
    'image_raw': tf.io.FixedLenFeature([], tf.string),
}

def _parse_function(example_proto, size, is_augmented=True):
    """ TODO: Update description!
    """
    parsed_example = tf.io.parse_single_example(example_proto, image_feature_description)
    image_string = parsed_example['image_raw']
    label = parsed_example['label']
    
    # Don't use tf.image.decode_image, or the output shape will be undefined
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.image.convert_image_dtype(image_decoded, tf.float32) # convert to float values in [0, 1]
    
    if is_augmented: # augmented here!
        h = parsed_example['height']
        w = parsed_example['width']
        image = tf.image.random_flip_left_right(image)        
        image = tf.image.random_crop(image, [h, w, 3])
        image = tf.image.random_brightness(image, max_delta=32.0 / 255.0)
        image = tf.image.random_saturation(image, lower=0.5, upper=1.5)
        image = tf.clip_by_value(image, 0.0, 1.0)                
      
   # resize image to fit the input size
    resized_image = tf.image.resize_with_pad(image, size, size)
    
    # scale to [-1,1]
    resized_image = (resized_image - 0.5)*2.0

    return resized_image, label

train_dataset = (tf.data.TFRecordDataset('data/PetImages/train_19855.tfrecord')
                     .map(lambda ep: _parse_function(ep, 224, True), num_parallel_calls=4)
                     .shuffle(1000)
                     .batch(32)
                     .prefetch(1)  # make sure you always have one batch ready to serve
                )

val_dataset = (tf.data.TFRecordDataset('data/PetImages/val_4969.tfrecord')
                     .map(lambda ep: _parse_function(ep, 224, False), num_parallel_calls=4)
                     .batch(32)
                     .prefetch(1)  # make sure you always have one batch ready to serve
                )

In [12]:
mobilenet_conv = keras.applications.mobilenet_v2.MobileNetV2(weights='imagenet', 
                                          include_top=False, 
                                          input_shape=(224, 224, 3))
# Freeze the layers except the last 4 layers
for layer in mobilenet_conv.layers:
    layer.trainable = False
mobilenet_conv.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalizationV1) (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

In [13]:
# Create the model
model = keras.models.Sequential()
# Add the vgg convolutional base model
model.add(mobilenet_conv)
# Add new layers
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(512, 1))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Conv2D(256, 1))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Conv2D(128, 1))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.GlobalAveragePooling2D())
model.add(keras.layers.Dense(2, activation='softmax'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
batch_normalization_v2_1 (Ba (None, 7, 7, 1280)        5120      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 512)         655872    
_________________________________________________________________
activation_3 (Activation)    (None, 7, 7, 512)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 7, 256)         131328    
_________________________________________________________________
activation_4 (Activation)    (None, 7, 7, 256)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 7, 7, 128)        

In [14]:
model.compile(optimizer=tf.optimizers.Adam(5e-6),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit_generator(train_dataset, 
                    epochs=1,
                    validation_data=val_dataset)

UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node mobilenetv2_1.00_224_1/Conv1/Conv2D}}]]
	 [[GroupCrossDeviceControlEdges_0/training_1/Adam/Adam/Const/_83]] [Op:__inference_keras_scratch_graph_25994]

In [8]:
# include the epoch in the file name. (uses `str.format`)
checkpoint_path = "experiments/mobilenet-based-cnn/cp-{epoch:04d}.ckpt"
model.save_weights(checkpoint_path.format(epoch=100))
model.save(experiments/mo)

## Load pretrained model for inferencing

In [6]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9174784105532219908
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 6222946923608792885
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 1443275224338555989
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6980065690
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8927031712676622291
physical_device_desc: "device: 0, name: GeForce GTX 1070 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]
